# Create Agents to Research and Write an Article

requirements
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

# these expect to find a .env file at the directory above the lesson.                                                                                                                     # the format for that file is (without the comment)                                                                                                                                       #API_KEYNAME=AStringThatIsTheLongAPIKeyFromSomeService
def load_env():
    _ = load_dotenv(find_dotenv())

def get_openai_api_key():
    load_env()
    openai_api_key = os.getenv("OPENAI_API_KEY")
    return openai_api_key

def get_serper_api_key():
    load_env()
    openai_api_key = os.getenv("SERPER_API_KEY")
    return openai_api_key


# break line every 80 characters if line is longer than 80 characters
# don't break in the middle of a word
def pretty_print_result(result):
  parsed_result = []
  for line in result.split('\n'):
      if len(line) > 80:
          words = line.split(' ')
          new_line = ''
          for word in words:
              if len(new_line) + len(word) + 1 > 80:
                  parsed_result.append(new_line)
                  new_line = word
              else:
                  if new_line == '':
                      new_line = word
                  else:
                      new_line += ' ' + word
          parsed_result.append(new_line)
      else:
          parsed_result.append(line)
  return "\n".join(parsed_result)


In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you run it again and again.

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
Artificial Intelligence (AI) continues to revolutionize industries and transform the way we live and work. In this blog article, we will explore the latest trends, key players, and noteworthy news in the field of AI, providing valuable insights for our readers.

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine Learning and Deep Learning advancements
- AI-powered automation in various industries
- Ethical co

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Trends, Players, and News

**Introduction**
Artificial Intelligence (AI) has been a hot topic in recent years, with advancements continually changing the landscape of various industries. In this blog article, we will delve into the latest trends, key players, and noteworthy news in the field of AI, offering valuable insights for our readers.

**Latest Trends in AI**
Machine Learning and Deep Learning have been at the forefront of AI advancements, allowing for more sophisticated algorithms and models. AI-powered automation is also making waves in industries such as manufacturing, healthcare, and finance, streamlining processes and increasing efficiency. However, with these advancements come ethical considerations and regulations that must be carefully navigated. The rise of AI in sectors like healthcare, finance, and cybersecurity is evident, with AI technologies being utilized for data analysis, fraud

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

**Introduction**
Artificial Intelligence (AI) has been a hot topic in recent years, with advancements continually changing the landscape of various industries. In this blog article, we will delve into the latest trends, key players, and noteworthy news in the field of AI, offering valuable insights for our readers.

**Latest Trends in AI**
Machine Learning and Deep Learning have been at the forefront of AI advancements, allowing for more sophisticated algorithms and models. AI-powered automation is also making waves in industries such as manufacturing, healthcare, and finance, streamlining processes and increasing efficiency. However, with these advancements come ethical considerations and regulations that must be carefully navigated. The rise of AI in sectors like healthcare, finance, and cybersecurity is evident, with AI technologies being utilized for data analysis, fraud detection, and personalized medicine.

**Key Players in the AI Industry**
Some of the key players in the AI industry include Google's DeepMind, known for its research in AI and machine learning. IBM Watson is another prominent player, offering AI-powered solutions for businesses. Amazon Web Services (AWS) and Microsoft Azure provide cloud-based AI services, making it easier for organizations to integrate AI into their operations. OpenAI, founded by Elon Musk and others, focuses on developing safe and beneficial AI technologies for the future.

**Noteworthy News in AI**
Recent breakthroughs in natural language processing have garnered attention, with AI models achieving human-like performance in tasks such as text generation and sentiment analysis. AI applications in self-driving cars and robotics are also progressing, with companies like Tesla and Boston Dynamics leading the way. Investments and partnerships in AI startups continue to grow, indicating a bright future for the industry. However, ethical dilemmas and bias in AI algorithms remain significant challenges that need to be addressed.

**Implications of AI on Society**
As AI technology advances, its impact on society becomes more pronounced. From job automation to privacy concerns, the implications of AI are far-reaching. It is crucial for businesses and policymakers to consider the ethical and social implications of AI development to ensure that it benefits society as a whole.

**Call to Action: Stay Informed and Engage with AI Community**
To stay updated on the latest developments in AI and connect with like-minded individuals, consider joining AI communities, attending conferences, and following industry news sources. By staying informed and engaging with the AI community, you can contribute to the responsible development and adoption of AI technologies.

In conclusion, Artificial Intelligence is poised to revolutionize industries and transform the way we live and work. By staying informed and actively participating in the AI community, we can harness the potential of AI for the betterment of society. Let's embrace the future of AI together.

In [14]:
topic = "Future of AI engineering job"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Future of AI engineering job.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: Future of AI Engineering Job

Audience Analysis:
Target Audience: 
- Current AI engineering professionals 
- Students pursuing a career in AI engineering 
- Hiring managers in tech companies 

Interests: 
- Latest trends and advancements in AI technology 
- Job prospects and opportunities in the AI engineering field 
- Skill development and training resources 
- Industry insights and expert opinions 

Pain Points: 
- Uncertainty about the future of AI engineering jobs 
- Lack of clarity on 

I now can give a great answer

Final Answer:

# The Future of AI Engineering Jobs: Navigating the Evolving Landscape

## Introduction

The field of AI engineering is rapidly evolving, with advancements in technology reshaping job roles and opportunities. It is crucial for professionals and aspiring individuals in this field to stay updated on the latest trends to remain competitive. As AI continues to infiltrate various industries, the demand for skilled professionals is on the rise, creating both challenges and opportunities for those in the field.

## Latest Trends in AI Engineering Jobs

Specialized roles in AI engineering are becoming increasingly prevalent, reflecting the diverse applications of AI technology. From machine learning experts to data scientists, the field offers a wide array of career paths for individuals with the right skills and qualifications. Automation is also playing a significant role in reshaping job roles, with repetitive tasks being automated to streamline

In [15]:
Markdown(result)

# The Future of AI Engineering Jobs: Navigating the Evolving Landscape

## Introduction

The field of AI engineering is rapidly evolving, with advancements in technology reshaping job roles and opportunities. It is crucial for professionals and aspiring individuals in this field to stay updated on the latest trends to remain competitive. As AI continues to infiltrate various industries, the demand for skilled professionals is on the rise, creating both challenges and opportunities for those in the field.

## Latest Trends in AI Engineering Jobs

Specialized roles in AI engineering are becoming increasingly prevalent, reflecting the diverse applications of AI technology. From machine learning experts to data scientists, the field offers a wide array of career paths for individuals with the right skills and qualifications. Automation is also playing a significant role in reshaping job roles, with repetitive tasks being automated to streamline processes and increase efficiency.

## Key Players in the Future of AI Engineering Jobs

Leading tech companies such as Google, Microsoft, and IBM are heavily investing in AI research and development, driving innovation and shaping the future of the industry. Additionally, emerging startups are disrupting the market with new AI applications and solutions, offering unique opportunities for professionals to explore. Academic institutions are also stepping up to meet the demand for AI engineering programs, providing students with the necessary skills to succeed in the field.

## Noteworthy News in AI Engineering Jobs

Recent advancements in AI technology, such as natural language processing and computer vision, are shaping the future job market and creating new opportunities for professionals. Industry collaborations and partnerships are driving innovation in AI engineering, leading to groundbreaking developments in various sectors. Job market predictions from industry experts suggest a continued growth in demand for AI professionals, highlighting the promising outlook for those in the field.

## Call to Action

As the landscape of AI engineering jobs continues to evolve, it is essential for professionals to stay informed and proactive in their career development. By continuously updating their skills and knowledge, individuals can position themselves for success in this dynamic field. Resources such as online courses, networking events, and job search platforms can help professionals navigate the competitive market and secure rewarding opportunities in AI engineering.

In conclusion, the future of AI engineering jobs is bright, with endless possibilities for growth and advancement. By staying informed, proactive, and adaptable, individuals in this field can navigate the ever-changing landscape of AI technology and carve out successful careers for themselves. Embracing the latest trends, learning from key players, and staying updated on industry news are essential steps towards building a thriving career in AI engineering.

Remember, the future belongs to those who are willing to adapt and innovate in the face of change. Keep learning, keep growing, and keep pushing the boundaries of what is possible in the exciting world of AI engineering.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```